In [1]:
# run initial imports
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from fbprophet import Prophet
from fbprophet.plot import plot_plotly

import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go

from scipy.integrate import odeint
from scipy.optimize import curve_fit

from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets

import folium
from folium.plugins import FastMarkerCluster

from datetime import datetime

In [2]:
NOW = datetime.now()
input_dir = './data/'
combined = pd.read_csv(input_dir + 'combined.csv')
combined['Last Update'] = pd.to_datetime(combined['Last Update'])
combined = combined.sort_values('Last Update').reset_index(drop=True)

#extract lat, lon for non mapped stuff
locations = combined.groupby(['Country/Region', 'Province/State'])['Latitude', 'Longitude'].mean().reset_index()
locations.columns = ['Country/Region', 'Province/State', 'Latitude_Lookup', 'Longitude_Lookup']

combined = pd.merge(left=combined, right=locations, left_on=['Country/Region', 'Province/State'], right_on=['Country/Region', 'Province/State'], how='left')
combined['Latitude'] = combined['Latitude'].fillna(combined['Latitude_Lookup'])
combined['Longitude'] = combined['Longitude'].fillna(combined['Longitude_Lookup'])
combined['Province/State'] = combined['Province/State'].fillna(combined['Country/Region'])
del combined['Latitude_Lookup'] 
del combined['Longitude_Lookup'] 
combined = combined.fillna(0)

In [3]:
#do a little re-ordering
combined = combined[['Latitude', 'Longitude', 'Last Update', 'Country/Region', 'Province/State','Confirmed','Recovered','Deaths']]
combined

,Latitude,Longitude,Last Update,Country/Region,Province/State,Confirmed,Recovered,Deaths
0,30.9756,112.2707,2020-01-22 17:00:00,China,Hubei,444.0,28.0,17.0
1,22.1667,113.5500,2020-01-22 17:00:00,Macau,Macau,1.0,0.0,0.0
2,37.2692,106.1655,2020-01-22 17:00:00,China,Ningxia,1.0,0.0,0.0
3,35.7452,95.9956,2020-01-22 17:00:00,China,Qinghai,0.0,0.0,0.0
4,35.1917,108.8701,2020-01-22 17:00:00,China,Shaanxi,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
7921,52.9399,-73.5491,2020-03-22 23:43:03,Canada,Quebec,219.0,0.0,5.0
7922,44.6820,-63.7443,2020-03-22 23:43:03,Canada,Nova Scotia,28.0,0.0,0.0
7923,53.9333,-116.5765,2020-03-22 23:43:03,Canada,Alberta,259.0,0.0,1.0
7924,7.9465,-1.0232,2020-03-22 23:43:03,Ghana,Ghana,24.0,0.0,1.0


In [4]:
latest = combined.sort_values('Last Update').groupby(['Latitude','Longitude']).tail(1).copy().reset_index(drop=True)
latest['Active'] = latest['Confirmed'] - latest['Recovered'] - latest['Deaths']

In [5]:
deaths = latest[latest['Deaths']>0]
deaths = pd.DataFrame(deaths.values.repeat(deaths.Deaths, axis=0), columns=deaths.columns)

confirmed = latest[latest['Confirmed']>0]
confirmed = pd.DataFrame(confirmed.values.repeat(confirmed.Confirmed, axis=0), columns=confirmed.columns)

us_confirmed = latest[latest['Confirmed']>0]
us_confirmed = us_confirmed[us_confirmed['Country/Region'] == 'US']
us_confirmed = pd.DataFrame(us_confirmed.values.repeat(us_confirmed.Confirmed, axis=0), columns=us_confirmed.columns)

recovered = latest[latest['Recovered']>0]
recovered = pd.DataFrame(recovered.values.repeat(recovered.Recovered, axis=0), columns=recovered.columns)

active = latest[latest['Active']>0]
active = pd.DataFrame(active.values.repeat(active.Active, axis=0), columns=active.columns)

us_active = latest[latest['Active']>0]
us_active = us_active[us_active['Country/Region'] == 'US']
us_active = pd.DataFrame(us_active.values.repeat(us_active.Active, axis=0), columns=us_active.columns)

na = latest[latest['Active'] > 0]
na_active = na[na['Country/Region'] == 'US']
na_active = na_active.append(na[na['Country/Region'] == 'Mexico'])
na_active = na_active.append(na[na['Country/Region'] == 'Canada'])
na_active = pd.DataFrame(na_active.values.repeat(na_active.Active, axis=0), columns=na_active.columns)

In [6]:
folium_map = folium.Map(location=[38.826555, -100.244867],
                        zoom_start=4,
                        tiles='CartoDB dark_matter')

                             
#FastMarkerCluster(data=list(zip(deaths['Latitude'].values, deaths['Longitude'].values)), name='Reported Deaths').add_to(folium_map)
#FastMarkerCluster(data=list(zip(recovered['Latitude'].values, recovered['Longitude'].values)), name='Recovered Cases').add_to(folium_map)
#FastMarkerCluster(data=list(zip(active['Latitude'].values, active['Longitude'].values)), name='Active Cases').add_to(folium_map)
FastMarkerCluster(data=list(zip(na_active['Latitude'].values, na_active['Longitude'].values)), name='North America Active Cases').add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map.save('./www/mapping.html')
folium_map